In [7]:
import pandas as pd
import numpy as np
import os

In [8]:
def get_csv(epoch):
    return pd.read_csv(f'outputs++/batches_permutations_model=BasicConv(in_channels=1, out_classes=2)_epoch={epoch}.csv')

In [11]:
N_EPOCHS = 4
dfs = [get_csv(e) for e in range(N_EPOCHS)]

In [25]:
for e in range(N_EPOCHS):
    dfs[e] = dfs[e].rename(columns={
        'FinalBatches': f'Epoch{e}Batches',
        'FinalBatchesIndexes': f'FinalBatchesIndexes_{e}',
        'FinalPermutationIndex': f'FinalPermutationIndex_{e}',
        'TestLoss': f'TestLoss_{e}',
        'TestCorrect': f'TestCorrect_{e}',
        'TestTotal': f'TestTotal_{e}',
        'TestAcc': f'TestAcc_{e}',
    })

In [37]:
merged = [dfs[0]]
for e in range(1, N_EPOCHS):
    new_merged = merged[e - 1].merge(dfs[e], how='inner', on=[f'Epoch{prev_epoch}Batches' for prev_epoch in range(e)])
    merged.append(new_merged)

In [66]:
def correlations(from_column, to_column, method='pearson'):
    print(f'Pearson correlation coefficients from column {from_column} to {to_column}:')
    for epoch in range(1, N_EPOCHS):
        also_check_against_same_epoch = 1 if from_column != to_column else 0
        for prev_epoch in range(0, epoch + also_check_against_same_epoch):
            m = merged[epoch]
            corr = m[f'{from_column}_{prev_epoch}'].corr(m[f'{to_column}_{epoch}'])
            print(f'{prev_epoch} -> {epoch}: {corr}')

In [67]:
correlations('TestLoss', 'TestLoss')

Pearson correlation coefficients from column TestLoss to TestLoss:
0 -> 1: 0.6871795552340948
0 -> 2: 0.8268248636252998
1 -> 2: 0.7771881458742893
0 -> 3: 0.24946785419223927
1 -> 3: 0.28847612325368366
2 -> 3: -0.036370899064337846


In [72]:
correlations('TestLoss', 'TestAcc')

Pearson correlation coefficients from column TestLoss to TestAcc:
0 -> 1: 0.09094643306554057
1 -> 1: -0.3877568461755002
0 -> 2: -0.057317081776166054
1 -> 2: -0.023018521093933632
2 -> 2: -0.028496220858925107
0 -> 3: -0.2201780857908947
1 -> 3: -0.1931862308796712
2 -> 3: -0.17395836541612233
3 -> 3: -0.5027137066795899


In [73]:
correlations('TestAcc', 'TestAcc')

Pearson correlation coefficients from column TestAcc to TestAcc:
0 -> 1: -0.7180258968952054
0 -> 2: -0.12965851753541172
1 -> 2: -0.35723846089139905
0 -> 3: -0.11191876812452033
1 -> 3: 0.2689754896046859
2 -> 3: -0.08864235299323987
